In [1098]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB


In [1099]:
data = pd.read_csv('C:\Th-th-ch-BAI-Labs\Bai_7\data\insurance_claims.csv')

<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\T'
C:\Users\Admin\AppData\Local\Temp\ipykernel_19936\3938155762.py:1: SyntaxWarning: invalid escape sequence '\T'
  data = pd.read_csv('C:\Th-th-ch-BAI-Labs\Bai_7\data\insurance_claims.csv')


In [1100]:
dropc_list = ['policy_number','policy_bind_date','policy_state','insured_zip','incident_location','incident_date',
           'incident_state','incident_city','insured_hobbies','auto_make','auto_model','auto_year', '_c39',
           'age','total_claim_amount']
dropr_list = [578]
df = data.copy()

df.drop(columns=dropc_list, inplace=True)
df.drop(index=dropr_list, inplace=True)
df['fraud_reported'] = df['fraud_reported'].replace({'Y':1, 'N':0})
target = df.pop('fraud_reported')
df.replace('?', np.nan, inplace = True)
df['collision_type'] = df['collision_type'].fillna(df['collision_type'].mode()[0])
df['property_damage'] = df['property_damage'].fillna(df['property_damage'].mode()[0])
df['police_report_available'] = df['police_report_available'].fillna(df['police_report_available'].mode()[0])
o_col = df.select_dtypes(include = ['object'])
num_df = df.select_dtypes(include = ['int64'])
o_col = pd.get_dummies(o_col)


scaler = StandardScaler()
name = num_df.columns
num_df = scaler.fit_transform(num_df)

num_df = pd.DataFrame(num_df, columns = name)
df = pd.concat([num_df, o_col], axis = 1)
df = df.dropna()
target = target.dropna()
target.drop(axis  = 0, index = 998,inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_19936\2053998201.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['fraud_reported'] = df['fraud_reported'].replace({'Y':1, 'N':0})


In [1101]:
df

,months_as_customer,policy_deductable,umbrella_limit,capital-gains,capital-loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,injury_claim,...,incident_severity_Total Loss,incident_severity_Trivial Damage,authorities_contacted_Ambulance,authorities_contacted_Fire,authorities_contacted_Other,authorities_contacted_Police,property_damage_NO,property_damage_YES,police_report_available_NO,police_report_available_YES
0,1.081258,-0.222499,-0.479771,1.012150,0.954763,-0.955411,-0.823305,0.009764,0.463400,-0.189102,...,False,False,False,False,False,True,False,True,False,True
1,0.211081,1.411889,1.696847,-0.900939,0.954763,-0.523791,-0.823305,-1.209559,-1.337910,-1.363058,...,False,False,False,False,False,True,True,False,True,False
2,-0.606885,1.411889,1.696847,0.358900,0.954763,-0.667664,1.139659,1.229087,1.364055,0.054704,...,False,False,False,False,False,True,True,False,True,False
3,0.454731,1.411889,2.132170,0.854221,-1.266566,-0.955411,-0.823305,0.009764,0.463400,-0.223931,...,False,False,False,False,False,True,True,False,True,False
4,0.211081,-0.222499,2.132170,1.467989,-0.682755,1.202688,-0.823305,-1.209559,-0.437255,-1.256521,...,False,False,False,False,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,-1.746817,-0.222499,-0.479771,-0.900939,0.954763,1.202688,-0.823305,-1.209559,-0.437255,2.050224,...,False,False,False,False,False,False,True,False,False,True
995,0.707082,-0.222499,-0.479771,1.643864,0.954763,1.634308,-0.823305,1.229087,1.364055,2.181347,...,False,False,False,True,False,False,False,True,True,False
996,-0.641692,-1.039694,0.826200,0.358900,0.954763,-1.099284,1.139659,1.229087,1.364055,0.013728,...,False,False,False,True,False,False,False,True,True,False
997,2.212488,1.411889,1.696847,-0.900939,0.954763,-1.387031,-0.823305,-1.209559,-0.437255,-0.453395,...,False,False,False,False,False,True,True,False,False,True


In [1102]:
X_train, X_test, y_train, y_test = train_test_split(df, target, test_size = 0.2, random_state = 42)

In [1103]:
pca = PCA(n_components=1 )
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)
print(pca.explained_variance_ratio_.sum())

0.13779044296185383


In [1104]:
rf = RandomForestClassifier(criterion= 'entropy',
                            min_samples_leaf= 1, 
                            min_samples_split = 10, 
                            n_estimators= 82, 
                            random_state = 42,
                            class_weight = 'balanced_subsample',)
rf.fit(X_train_pca, y_train)
dt = DecisionTreeClassifier(max_depth = 1,
                            min_samples_leaf= 3, 
                            min_samples_split = 3, 
                            random_state = 42,)
dt.fit(X_train_pca, y_train)
xgb = XGBClassifier()
xgb.fit(X_train_pca, y_train)
gbc = GradientBoostingClassifier()
gbc.fit(X_train_pca, y_train)
gnb = GaussianNB()
gnb.fit(X_train_pca, y_train)
knn = KNeighborsClassifier()
knn.fit(X_train_pca, y_train)

,n_neighbors,5
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [1105]:
y_pred_rf = rf.predict(X_test_pca)
y_pred_xgb = xgb.predict(X_test_pca)
y_pred_gbc = gbc.predict(X_test_pca)
y_pred_gnb = gnb.predict(X_test_pca)
y_pred_knn = knn.predict(X_test_pca)
y_pred_dt = dt.predict(X_test_pca)

In [1106]:
print(f1_score(dt.predict(X_train_pca), y_train))
print(f1_score(dt.predict(X_test_pca), y_test))

0.0
0.0


In [1107]:
print(f1_score(rf.predict(X_train_pca), y_train))
print(f1_score(rf.predict(X_test_pca), y_test))

0.7708333333333334
0.3157894736842105


In [1108]:
print(f1_score(y_pred_rf, y_test))
print(f1_score(y_pred_xgb, y_test))
print(f1_score(y_pred_gbc, y_test))
print(f1_score(y_pred_gnb, y_test))
print(f1_score(y_pred_knn, y_test))

0.3157894736842105
0.11940298507462686
0.0967741935483871
0.0
0.16666666666666666
